# DAY 3: Student version

**Machine Learning NLP**

The goal of this session is to improve the search engine using NLP features.

This notebook guides you through different techniques to explore. It is expected of you to be inventive and improve the techniques introduced. 

First, let's import the useful packages and load the data.

## Installs

In [2]:
! pip install sentence-transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 113.7 MB/s eta 0:00:00


## Imports

In [3]:
import os
import re
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from scipy.sparse import find
from sklearn.metrics.pairwise import cosine_similarity

from sentence_transformers import SentenceTransformer

import nltk
nltk.download('punkt')
nltk.download('wordnet')

from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


## Load Data

In [4]:
# Only if you use Colab
from google.colab import drive
drive.mount('/content/drive')

import os

# TODO:
DATA_PATH = '' 

# CORR:
DATA_PATH = '/content/drive/MyDrive/EI/Dataset/data'
     

Mounted at /content/drive


In [5]:
posts = pd.read_xml(os.path.join(DATA_PATH, 'Posts.xml'), parser="etree", encoding="utf8")
posts

,Id,PostTypeId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastActivityDate,Title,Tags,...,ClosedDate,ContentLicense,AcceptedAnswerId,LastEditorUserId,LastEditDate,ParentId,OwnerDisplayName,CommunityOwnedDate,LastEditorDisplayName,FavoriteCount
0,5,1,2014-05-13T23:58:30.457,9,898.0,<p>I've always been interested in machine lear...,5.0,2014-05-14T00:36:31.077,How can I do simple machine learning without h...,<machine-learning>,...,2014-05-14T14:40:25.950,CC BY-SA 3.0,NaN,NaN,None,NaN,None,None,None,NaN
1,7,1,2014-05-14T00:11:06.457,4,478.0,"<p>As a researcher and instructor, I'm looking...",36.0,2014-05-16T13:45:00.237,What open-source books (or other materials) pr...,<education><open-source>,...,2014-05-14T08:40:54.950,CC BY-SA 3.0,10.0,97.0,2014-05-16T13:45:00.237,NaN,None,None,None,NaN
2,9,2,2014-05-14T00:36:31.077,5,NaN,"<p>Not sure if this fits the scope of this SE,...",51.0,2014-05-14T00:36:31.077,None,None,...,None,CC BY-SA 3.0,NaN,NaN,None,5.0,None,None,None,NaN
3,10,2,2014-05-14T00:53:43.273,13,NaN,"<p>One book that's freely available is ""The El...",22.0,2014-05-14T00:53:43.273,None,None,...,None,CC BY-SA 3.0,NaN,NaN,None,7.0,None,None,None,NaN
4,14,1,2014-05-14T01:25:59.677,26,1901.0,<p>I am sure data science as will be discussed...,66.0,2020-08-16T13:01:33.543,Is Data Science the Same as Data Mining?,<data-mining><definitions>,...,None,CC BY-SA 3.0,29.0,322.0,2014-06-17T16:17:20.473,NaN,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75722,119962,1,2023-03-04T20:06:06.820,0,8.0,<p>I am implementing a neural network of arbit...,147597.0,2023-03-04T20:22:12.523,Back Propagation on arbitrary depth network wi...,<neural-network><backpropagation>,...,None,CC BY-SA 4.0,NaN,147597.0,2023-03-04T20:22:12.523,NaN,None,None,None,NaN
75723,119963,1,2023-03-04T20:12:19.677,0,10.0,<p>I am using KNN for a regression task</p>\n<...,147598.0,2023-03-04T20:12:19.677,Evaluation parameter in knn,<regression><k-nn>,...,None,CC BY-SA 4.0,NaN,NaN,None,NaN,None,None,None,NaN
75724,119964,1,2023-03-05T00:14:12.597,0,7.0,<p>I have developed a small encoding algorithm...,44581.0,2023-03-05T00:14:12.597,Can I use zero-padded input and output layers ...,<deep-learning><convolutional-neural-network>,...,None,CC BY-SA 4.0,NaN,NaN,None,NaN,None,None,None,NaN
75725,119965,1,2023-03-05T00:43:12.213,0,5.0,"<p>To my understanding, optimizing a model wit...",84437.0,2023-03-05T00:43:12.213,Why does cross validation and hyperparameter t...,<cross-validation><hyperparameter-tuning>,...,None,CC BY-SA 4.0,NaN,NaN,None,NaN,None,None,None,NaN


## Data Cleaning

Implement a function to clean the posts. 

You can reuse what you have used in the Day 2 notebook or improve it.

In [6]:
def clean_post(text:str)->str:
  res=""
  in_tag=False
  next=False
  for lettre in text:
    if lettre=='<' and not in_tag:
      in_tag=True
    elif lettre=='>' and in_tag:
      in_tag=False
    elif lettre=='\n' or lettre=='\t':
      next=True
    elif not in_tag:
      if next:
        res+=" " + str(lettre)
        next=False
      else:
        res+=str(lettre) 
  return res


In [7]:
clean_posts = posts[['Id','Body']]
clean_posts['Clean_Body'] = clean_posts['Body'].fillna('').apply(clean_post)
clean_posts

<ipython-input-7-c00d1db547f3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_posts['Clean_Body'] = clean_posts['Body'].fillna('').apply(clean_post)


,Id,Body,Clean_Body
0,5,<p>I've always been interested in machine lear...,I've always been interested in machine learnin...
1,7,"<p>As a researcher and instructor, I'm looking...","As a researcher and instructor, I'm looking fo..."
2,9,"<p>Not sure if this fits the scope of this SE,...","Not sure if this fits the scope of this SE, bu..."
3,10,"<p>One book that's freely available is ""The El...","One book that's freely available is ""The Eleme..."
4,14,<p>I am sure data science as will be discussed...,I am sure data science as will be discussed in...
...,...,...,...
75722,119962,<p>I am implementing a neural network of arbit...,I am implementing a neural network of arbitrar...
75723,119963,<p>I am using KNN for a regression task</p>\n<...,I am using KNN for a regression task It's like...
75724,119964,<p>I have developed a small encoding algorithm...,I have developed a small encoding algorithm th...
75725,119965,"<p>To my understanding, optimizing a model wit...","To my understanding, optimizing a model with k..."


You can also implement a function that cleans the user's query (the query). 

This step is optionnal (if you don't think that it is necessary, just return the query)

In [8]:
def clean_query(text:str)->str:
    cleaned_query = clean_post(text)
    return cleaned_query

## Text specific metadata

What metadata can you get from the text at your disposal ? Which ones are relevant ? 

In [9]:
posts.keys()

Index(['Id', 'PostTypeId', 'CreationDate', 'Score', 'ViewCount', 'Body',
       'OwnerUserId', 'LastActivityDate', 'Title', 'Tags', 'AnswerCount',
       'CommentCount', 'ClosedDate', 'ContentLicense', 'AcceptedAnswerId',
       'LastEditorUserId', 'LastEditDate', 'ParentId', 'OwnerDisplayName',
       'CommunityOwnedDate', 'LastEditorDisplayName', 'FavoriteCount'],
      dtype='object')

## Classic Preprocessing

The goal for this part is to implement a classic vectorization (Count vectorizer, tfidf...).

You can do it on your own or use scikit-learn.

Hints : pay attention to stopwords, additionnal preprocessing steps and techniques of vectoriation


In [10]:
vectorizer = CountVectorizer(stop_words = 'english')
vectorizer.fit(clean_posts.Clean_Body.values)
vectors = vectorizer.transform(clean_posts.Clean_Body.values)
vectors

<75727x180787 sparse matrix of type '<class 'numpy.int64'>'
	with 3862609 stored elements in Compressed Sparse Row format>

In [11]:
vectors

<75727x180787 sparse matrix of type '<class 'numpy.int64'>'
	with 3862609 stored elements in Compressed Sparse Row format>

Write a function that applies the same process to the query


In [12]:
def vectorize_query(query : str, vectorizer=vectorizer):
    """vectorizes the query
    Args:
        query (str): query string
        vectorizer (optional): Defaults to vectorizer.

    Returns:
        query vectorized
    """
    cleaned_query = clean_query(query)
    query_vectorized = vectorizer.transform([cleaned_query])

    return query_vectorized

In [13]:
query = vectorize_query("This is a test query about penguins")

Determine a way to use this vectorization to suggest the closest items to the entry in the database

In [14]:
def vectorizer_search(query : str,
                      vectors=vectors,
                      vectorizer=vectorizer,
                      top = 5) -> list:
  vectorized_query = vectorize_query(query)
  similarities = cosine_similarity(vectors, vectorized_query)
  similarities = [[similarities[k][0], k] for k in range(len(similarities))]
  similarities.sort(reverse = True)
  closest_items = [clean_posts['Clean_Body'][x[1]] for x in similarities[:top]]
  return closest_items

In [15]:
entry = 'what is stochastic gradient descent ?'

In [16]:
 vectorizer_search(entry, top = 10)

['As I know, Gradient Descent has three variants which are: 1- Batch Gradient Descent: processes all the training examples for each iteration of gradient descent. 2- Stochastic Gradient Descent: processes one training example per iteration. Hence, the parameters are being updated even after one iteration in which only a single example has been processed. 3- Mini Batch gradient descent: which works faster than both batch gradient descent and stochastic gradient descent. Here, b examples where b &lt; m are processed per iteration. But in some cases they use Stochastic Gradient Descent and they define a batch size for training which is what I am confused about. Also, what about Adam, AdaDelta &amp; AdaGrad, are they all mini-batch gradient descent or not?',
 'What is the difference between Gradient Descent and Stochastic Gradient Descent?  I am not very familiar with these, can you describe the difference with a short example?',
 "I am a student and I am studying machine learning. I am fo

How you can improve this approach ? 

Answer here

## Semantic similarity

There are NLP methods that go further than word-by-word study, by taking into account the context of the terms. There are several methods: Word2vec, Bert.

From the Sentence Transformers documentation: https://www.sbert.net/docs/pretrained_models.html choose the pre-trained model that you think is the most appropriate. Justify your choice.

In [17]:
sentence_transformer_model = 'multi-qa-MiniLM-L6-cos-v1'

Answer Here

⚠ To use Sentence Transformers it is recommended to activate the GPU of google colab.

In [18]:
MODEL_ST = SentenceTransformer(sentence_transformer_model)

Use this algorithm to encode the data in the database

In [19]:
embeddings = MODEL_ST.encode(clean_posts.Clean_Body.values, normalize_embeddings=True)

In [26]:
from numpy import savetxt
savetxt('embeddings.pkl', embeddings, delimiter = ',')

*If this process is slow, you can save this array in case you need to load it again*

In [22]:
import pickle

with open(os.path.join(DATA_PATH, 'embeddings.pkl', 'wb')) as file:
    pickle.dump(embeddings, file)

FileNotFoundError: ignored

Make a function that transforms the input

In [ ]:
def encode_query(query : str) ->  numpy.ndarray:
    #TODO
    return encoded_query

Which distance is most relevant to measure the distance between the input and the data?

Answer here

Write a function that returns a matrix containing information about the similarity between the query and the data

In [ ]:
def similarity(query, embeddings=embeddings):
    #TODO
    return similarity_matrix

In [ ]:
matrix_similarity = similarity(query, embeddings)

How do you determine which documents in the data set most closely match the input?

In [ ]:
def ordre_en_fonction_similarité(matrix_similarity):
    #TODO
    return ordre

In [ ]:
assert ordre_en_fonction_similarité([0.6, 0.8, 0.7]) == [1, 2, 0]

[1, 2, 0]

Put it all together in a function.

In [ ]:
def closest_semantic_doc(query, embeddings=embeddings, top_n=10):
    return closest_posts

What methods could be used to improve the recommendations of this algorithm?

Answer here

In [ ]:
#todo

## Text clustering (BONUS)

We can use topic modeling techniques to identify groups of texts among our document base and classify the input to restrict the application of the proximity calculations seen previously.

#### LDA

Latent Dirichlet Allocation is a topic modeling algorithm that allows soft clustering. Soft clustering means that the LDA does not allocate an input to a cluster, but gives a probabilistic score for each identified cluster. This decomposition allows to identify topics within the documents. 

In order to compute this algorithm, you need to vectorize your data (you can use the one you have already done previously or make another one).

In [ ]:
# Vectorize document using TF-IDF
vectorizer_lda = 

# Fit and Transform the documents
train_data = vectorizer_lda.fit_transform(posts.cleaned_body.values)

You can use Gensim or scikit-learn to compute LDA.

In [ ]:
#TODO

Assign a main topic to each document

In [ ]:
topic_documents = 

Write a function that assigns a topic to the query


In [ ]:
def get_topic_query(query, vectorizer=vectorizer_lda, lda_model=lda_model) -> int:
    #TODO
    return topic_query

## Merge methods

Write an algorithm to merge the methods seen. Which methods to use? How can you check if they are relevant ?

Answer here

In [ ]:
def nlp_search_algorithm(query,
                         topic_documents=topic_documents,
                         vectors=vectors,
                         vectorizer=vectorizer,
                         vectorizer_lda=vectorizer_lda,
                         lda_model=lda_model,
                         embeddings=embeddings,
                         top_n=10
                         )->list:
    #TODO
 
    return matching_posts




Once you have a list of possible results, you can it: (you can use one of the ranking algorithms you have previously done or make up a new one)

In [ ]:
def rank(possible_results):
    #to_do
    return possible_results

## Incorporation in the search engine

### Addition of metadata

You must now have a new set of metadata and data to add to your original index. You can load the index you had as a result of Day 2 and today's work to it. 

In [ ]:
#load previous data 

#TODO 

In [ ]:
# add the new data to the previous index

Hint : if you have changed the preprocessing function at the beginning of this notebook make sure to update the Clean Body attribute

### Adaptation to the index format

Adapt your nlp search results to the index format

In [ ]:
def nlp_search_in_index(query,
                        index,
                        args):

    return matching_posts
  

### Compare the new searching and ranking method to the previous ones

Compare in terms of efficiency (precision, completeness, speed, memory usage)

### merge all methods to make an efficient search algorithm